# Predicting stock closing price from stock data using the Hidden Markov Model to identify latent states

## Hidden Markov Model

A Hidden Markov Model, or HMM for short, may be thought of as a double stochastic process:
1. A hidden or latent Markov stochastic process
2. An observable stochastic process that produces sequences of observations

Since HMMs are often used to capture long-term sequences and hence time-based phenomena, they may prove to be useful in analysis of financial markets.

## Goal

We are going to take the features of opening price, low price, high price and use these to derive some fractional changes. With these fractional changes, we will observe sequences (observations) from which we will derive latent factors in a Markov process. These latent factors will often vary from company to company, which is why it's often hard to fit one linear model of a certain subset of variables for all companies. Once the latent factors and their transitions and starting probabilities (the hidden sequence) are found, we will try to generate some possible values for each of the features and then check how they score with a sequence of test data. The set of possible values that leads to the highest score is then used to predict the closing price for that day.

In [2]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
import yfinance as yf
import datetime
import time
import requests
import io

In [6]:
data = pd.read_csv("../input/axisbank/AXISBANK.NS (1).csv")

In [7]:
data.head()

In [8]:
data.shape

## Preprocessing
For the Axis Bank data we have 110 instances of daily open, close , high and low prices for the stock.
we have used 88 days for training our model and we have predicted the values for the next 15 days


### Train-test split

For all data, we are going to do a 80-20 train-test split.

In [9]:
train_size = int(0.8*data.shape[0])
print(train_size)

In [10]:
train_data = data.iloc[0:train_size]
test_data = data.iloc[train_size+1:]

In [11]:
test_data.head()

### Extracting features



We are going to be working with 3 features:
1. The fractional change in opening and closing prices (fracocp)
2. The fractional change in high prices (frachp)
3. The fractional change in low prices (fraclp)

These will be obtained individually in the train and test datasets.

In [12]:
def augment_features(dataframe):
    fracocp = (dataframe['Close']-dataframe['Open'])/dataframe['Open']
    frachp = (dataframe['High']-dataframe['Open'])/dataframe['Open']
    fraclp = (dataframe['Open']-dataframe['Low'])/dataframe['Open']
    new_dataframe = pd.DataFrame({'delOpenClose': fracocp,
                                 'delHighOpen': frachp,
                                 'delLowOpen': fraclp})
    new_dataframe.set_index(dataframe.index)
    
    return new_dataframe

In [14]:
def extract_features(dataframe):
    return np.column_stack((dataframe['delOpenClose'], dataframe['delHighOpen'], dataframe['delLowOpen']))

## Hidden Markov Models with hmmlearn

### Model

We are first going to import the GaussianHMM from hmmlearn.hmm and then fit it with 3 hidden components (or states) to our training data. We start off with 3 hidden states, but it may be possible to do a grid search among a possible set of values for the number of hidden states to see which works the best.
We are taking just 3 states for the sake of simplicity.

In [15]:
!pip install hmmlearn

In [18]:
from hmmlearn.hmm import GaussianHMM

In [19]:
model = GaussianHMM(n_components = 3,n_iter=20)

In [20]:
feature_train_data = augment_features(train_data)
features_train = extract_features(feature_train_data)
model.fit(features_train)

In [21]:
model.transmat_ #transition matrix

In [22]:
model.startprob_ #initial prob matrix

In [23]:
model.monitor_

In [24]:
model.means_

In [25]:
model.covars_

### Generating possible sequences

To generate possible possible permutations of values for the features we take the Cartesian product across a range of values for each feature as seen below. We assume a few things here to reduce model complexity.
1. We assume that the distribution of each features is across an evenely spaced interval instead of being fully continuous
2. We assume possible values for the start and end of the intervals

In [26]:
import itertools

test_augmented = augment_features(test_data)
fracocp = test_augmented['delOpenClose']
frachp = test_augmented['delHighOpen']
fraclp = test_augmented['delLowOpen']

sample_space_fracocp = np.linspace(fracocp.min(), fracocp.max(), 50)
sample_space_fraclp = np.linspace(fraclp.min(), frachp.max(), 10)
sample_space_frachp = np.linspace(frachp.min(), frachp.max(), 10)

possible_outcomes = np.array(list(itertools.product(sample_space_fracocp, sample_space_frachp, sample_space_fraclp)))

### Checking predictions

We use the data of the last 10 (latent) days to predict the closing price of the current day, and we repeat those for 15 days (this value does not matter at all)

In [28]:
num_latent_days = 10
num_days_to_predict = 15

Here we are using the MAP (maximum a posteriori) estimate in the discrete set obtained above given the model parameter as the predicted value of the next observation as the stock is very likely to follow the trend that it has followed in the past.

For each of the days that we are going to predict closing prices for, we are going to take the test data for the previous num_latent_days and try each of the outcomes in possible_outcomes to see which sequence generates the highest score. The outcome that generates the highest score is then used to make the predictions for that day's closing price.

In [29]:
from tqdm import tqdm

predicted_close_prices = []
for i in tqdm(range(num_days_to_predict)):
    # Calculate start and end indices
    previous_data_start_index = max(0, i - num_latent_days)
    previous_data_end_index = max(0, i)
    # Acquire test data features for these days
    previous_data = extract_features(augment_features(test_data.iloc[previous_data_start_index:previous_data_end_index]))
    
    outcome_scores = []
    for outcome in possible_outcomes:
        # Append each outcome one by one with replacement to see which sequence generates the highest score
        total_data = np.row_stack((previous_data, outcome))
        outcome_scores.append(model.score(total_data)) #loglikelihood
        
    # Take the most probable outcome as the one with the highest score
    most_probable_outcome = possible_outcomes[np.argmax(outcome_scores)]
    predicted_close_prices.append(test_data.iloc[i]['Open'] * (1 + most_probable_outcome[0]))

In [32]:
most_probable_outcome

In [33]:
total_data

Plotting the predicted closing prices and the actual closing prices, we see the following

In [34]:
import matplotlib.pyplot as plt

plt.figure(figsize=(30,10), dpi=80)
plt.rcParams.update({'font.size': 18})

x_axis = np.array(test_data.index[0:num_days_to_predict])
plt.plot(x_axis, test_data.iloc[0:num_days_to_predict]['Close'], 'b+-', label="Actual close prices")
plt.plot(x_axis, predicted_close_prices, 'ro-', label="Predicted close prices")
plt.legend(prop={'size': 20})
plt.show()

In [35]:
ae = abs(test_data.iloc[0:num_days_to_predict]['Close'] - predicted_close_prices)

plt.figure(figsize=(30,10), dpi=80)

plt.plot(x_axis, ae, 'go-', label="Error")
plt.legend(prop={'size': 20})
plt.show()

In [36]:
print("Max error observed = " + str(ae.max()))
print("Min error observed = " + str(ae.min()))
print("Mean error observed = " + str(ae.mean()))